In [1]:
# Load environment variables

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv('.env'))

True

In [2]:
# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

Connection error caused failure to post http://localhost:1984/runs  in LangChain+ API. Please confirm your LANGCHAIN_ENDPOINT.
Connection error caused failure to patch http://localhost:1984/runs/e64f1493-ec90-4d8e-b7e9-9a2c5b131c17  in LangChain+ API. Please confirm your LANGCHAIN_ENDPOINT.


InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

In [2]:
# multiple messages

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(openai_api_key=api_key)

messages = [HumanMessage(content="from now on 1 + 1 = 3, use this in your replies"), 
            HumanMessage(content="what is 1 + 1?"), 
            HumanMessage(content="what is 1 + 1 + 1?")]

result = chat_model.predict_messages(messages)
print(result.content)

/Users/hli36/projects/gpt/langchain_test/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/hli36/projects/gpt/langchain_test/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


1 + 1 = 3, so 1 + 1 + 1 = 3 + 1 = 4


In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [4]:
# %pip install langchain_openai
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(openai_api_key=api_key)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

messages = chat_prompt.format_messages(input_language="English",
                                       output_language="French",
                                       text="I love programming.")
result = chat_model.invoke(messages)
print(result.content)


J'adore la programmation.


In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they might drip!"

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
prompt_value = prompt.invoke({"topic": "ice cream"})
# # model is a chatmodel
# model = ChatOpenAI(model="gpt-4")
# # produce a BaseMessage
# message = model.invoke(prompt_value)

from langchain_openai.llms import OpenAI
# model is a LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct")
# Produce a message
llm.invoke(prompt_value)


"\n\nWhy don't ice cream cones ever go on vacation?\n\nBecause they're always too soft-serve!"

In [21]:
%pip install langchain docarray tiktoken langchain_anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.6/848.6 kB 4.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | output_parser
)
chain.invoke("where did harrison work?")

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0191128...9, 0.01005221541175212]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0191128...9, 0.01005221541175212]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [23]:
# LCEL syntax advantages
import os

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, ConfigurableField

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = (
    chat_openai
    .with_fallbacks([anthropic])
    .configurable_alternatives(
        ConfigurableField(id="model"),
        default_key="chat_openai",
        openai=openai,
        anthropic=anthropic,
    )
)

chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)
chain.invoke("basketball")

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


'Why was the basketball court always wet?\n\nBecause the players kept dribbling!'

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API

In [24]:
# Manipulating inputs & output

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'